In [1]:
#aysenur

In [2]:
import pandas as pd
import numpy as np

In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning.metrics import F1
from pytorch_lightning.loggers import WandbLogger

from pytorch_lightning.loggers import TensorBoardLogger

In [4]:
from DataPreparation import get_data

In [5]:
CURRENCY_LST = ['BTC', 'ETH']#, 'LTC']
FREQUENCY = "D"

WINDOW_SIZE = 50
NEUTRAL_QUANTILE = 0.33 # çok olduğunu düşünüyorum
N_CLASSES = 3

In [6]:
TRAIN_PERCENTAGE, VAL_PERCENTAGE, TEST_PERCENTAGE = 0.97, 0.007, 0.023

In [7]:
LSTM_HIDDEN_SIZES = [128, 128, 128]

In [8]:
LOSS_WEIGHT_CALCULATE = False

In [9]:
BATCH_SIZE= 16

In [10]:
X, y, features, dfs = get_data(CURRENCY_LST,
                                N_CLASSES,
                                 FREQUENCY, 
                                 WINDOW_SIZE,
                                 neutral_quantile = NEUTRAL_QUANTILE,
                                 log_price=True,
                                 remove_trend=False,
                                 include_indicators = False,
                                 include_imfs = False
                                )

In [11]:
N_CURRENCIES = X.shape[0]
INPUT_FEATURE_SIZE = X.shape[-1]

N_CURRENCIES, INPUT_FEATURE_SIZE

(2, 1)

In [12]:
class MultiTimeSeriesDataset(Dataset):
    def __init__(self, 
                 n_currencies,
                 x: np.ndarray, 
                 y: np.ndarray,
                 data_use_type,
                 train_percentage = TRAIN_PERCENTAGE,
                 val_percentage = VAL_PERCENTAGE,
                 test_percentage = TEST_PERCENTAGE,
                 seq_len = WINDOW_SIZE, 
                 ):
        
        self.x = torch.tensor(x[:n_currencies]).float()
        self.y = torch.tensor(y[:n_currencies]).long()
        self.seq_len = seq_len
        self.data_use_type = data_use_type
        
        #self.train_size = int(len(self.x[0]) * train_percentage)
        self.val_size = int(len(self.x[0]) * val_percentage)
        self.test_size = int(len(self.x[0]) * test_percentage)
        self.train_size = len(self.x[0]) - self.val_size - self.test_size 
        
        self.train_mean = [self.x[i][:self.train_size].mean() for i in range(n_currencies)]
        self.train_std = [self.x[i][:self.train_size].std() for i in range(n_currencies)]
        
        self.train_min = [self.x[i][:self.train_size].min() for i in range(n_currencies)]
        self.train_max = [self.x[i][:self.train_size].max() for i in range(n_currencies)]
        
    def __len__(self):
        
        if self.data_use_type == "train":
            return self.train_size - ( self.seq_len)

        elif self.data_use_type == "val":
            return self.val_size
  
        else:
            return self.test_size
        
    
    def __getitem__(self, index):
        
        item = dict()
        
        if self.data_use_type =="val":
            index = self.train_size + index - self.seq_len
            
        elif self.data_use_type =="test":
            index = self.train_size + self.val_size + index - self.seq_len
        
        for i in range(N_CURRENCIES):
            window = self.x[i][index:index+self.seq_len]
            window = (window -self.train_mean[i]) / self.train_std[i]
            
            item["currency_" + str(i) + "_window"] = window
            item["currency_" + str(i) + "_label"]  = self.y[i][index+self.seq_len]

        return item

In [13]:
train_dataset, val_dataset, test_dataset = [MultiTimeSeriesDataset(N_CURRENCIES, X, y, dtype) for dtype in ['train', 'val', 'test']]

In [14]:
len(train_dataset), len(val_dataset), len(test_dataset)

(1269, 9, 31)

In [15]:
if LOSS_WEIGHT_CALCULATE:
    loss_weights = []
    for n in range(N_CURRENCIES):
        train_labels = [int(train_dataset[i]["currency_"+ str(n)+"_label"] )for i in range(train_dataset.__len__())]
        samples_size = pd.DataFrame({"label": train_labels}).groupby("label").size().to_numpy()
        print(samples_size)
        loss_weights.append((1 / samples_size) * sum(samples_size)/2)
    loss_weights
else:
    loss_weights = None

loss_weights

In [16]:
class LSTM_based_classification_model(pl.LightningModule):
    def __init__(self,
                 train_dataset = train_dataset,
                 val_dataset = val_dataset,
                 test_dataset = test_dataset,
                 weights = loss_weights,
                 num_tasks = N_CURRENCIES,
                 num_classes = N_CLASSES,
                 window_size = WINDOW_SIZE,
                 input_size = INPUT_FEATURE_SIZE,
                 batch_size=BATCH_SIZE,
                 lstm_hidden_sizes = LSTM_HIDDEN_SIZES,
                 bidirectional = False,
                 ):
        
        super().__init__()
        self.num_classes = num_classes
        self.num_tasks = num_tasks
        self.window_size = window_size
        self.input_size = input_size
        self.batch_size = batch_size
        
        self.lstm_hidden_sizes = lstm_hidden_sizes
        self.bidirectional = bidirectional 
        self.weights = weights
        
        
        self.lstm_1 = nn.LSTM(input_size = self.input_size, num_layers=1, batch_first=True, hidden_size = self.lstm_hidden_sizes[0])
        self.batch_norm1 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[0])
        
        self.lstm_2 = nn.LSTM(input_size = self.lstm_hidden_sizes[0], num_layers=1, batch_first=True, hidden_size = self.lstm_hidden_sizes[1])
        self.batch_norm2 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[1])
        
        self.lstm_3 = nn.LSTM(input_size = self.lstm_hidden_sizes[1], num_layers=1, batch_first=True, hidden_size = self.lstm_hidden_sizes[2])
        self.batch_norm3 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[2])
        
        self.dropout = nn.Dropout(0.5)
        
        #self.linear1 = nn.Linear(self.lstm_hidden_sizes[2], int(self.lstm_hidden_sizes[2]/2))
        self.linear1 =[nn.Linear(self.lstm_hidden_sizes[2], int(self.lstm_hidden_sizes[2]/2))] * self.num_tasks
        self.linear1 = torch.nn.ModuleList(self.linear1)
        self.activation = nn.ReLU()
        
        self.output_layers = [nn.Linear(int(self.lstm_hidden_sizes[2]/2), self.num_classes)] * self.num_tasks
        self.output_layers = torch.nn.ModuleList(self.output_layers)
        
        
        #self.cross_entropy_loss = nn.CrossEntropyLoss(weight= torch.tensor(weights).float()) # loss weight
        #self.cross_entropy_loss = nn.CrossEntropyLoss()
        
        if self.weights != None:
            self.cross_entropy_loss = [nn.CrossEntropyLoss(weight= torch.tensor(weights).float()) for weights in self.weights]
        else:
            self.cross_entropy_loss = [nn.CrossEntropyLoss() for _ in range(self.num_tasks)]
        
        self.cross_entropy_loss = torch.nn.ModuleList(self.cross_entropy_loss)
        
        self.f1_score = pl.metrics.F1(num_classes=self.num_classes, average="macro")
        self.accuracy_score = pl.metrics.Accuracy()
        
        self.train_dl = DataLoader(train_dataset, batch_size=self.batch_size, shuffle = True)
        self.val_dl = DataLoader(val_dataset, batch_size=self.batch_size)
        self.test_dl = DataLoader(test_dataset, batch_size=self.batch_size)
        
    def forward(self, x, i):

        batch_size = x.size()[0]
        
        x = x.view(batch_size, self.window_size, self.input_size) #(batch, window_len, feature_size)
        x, _  = self.lstm_1(x)
        
        x = self.dropout(x)

        x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
        x = self.batch_norm1(x.unsqueeze(0))

        x = x.view(batch_size, self.window_size, x.size()[1])
        x, _  = self.lstm_2(x)
        
        x = self.dropout(x)
        
        x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
        x = self.batch_norm2(x.unsqueeze(0))
        
        x = x.view(batch_size, self.window_size, x.size()[1])
        x, _  = self.lstm_3(x)
        
        x = self.dropout(x)
        
        x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
        x = self.batch_norm3(x.unsqueeze(0))
        
        x = x.view(batch_size, self.window_size, x.size()[1])
        x = x[:, -1, :] # equivalent to return sequence = False on keras :)
        
        x = self.dropout(x)
        
        x = self.linear1[i](x)
        x = self.activation(x)
                 
        output = self.output_layers[i](x)
    
        return output
    
    
    def training_step(self, batch, batch_nb):
        
        loss = (torch.tensor(0.0, device="cuda:0", requires_grad=True) + \
                torch.tensor(0.0, device="cuda:0", requires_grad=True)) 
        # araştırılabilir
        for i in range(self.num_tasks):
            x, y = batch["currency_" + str(i) + "_window"], batch["currency_" + str(i) + "_label"]

            output = self.forward(x, i)
            #loss = F.nll_loss(output, y)
            loss += self.cross_entropy_loss[i](output, y)
            
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log("currency_" + str(i) +'_train_acc', acc, on_epoch=True, prog_bar=True)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log("currency_" + str(i) +'_train_f1', f1, on_epoch=True, prog_bar=True)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        
        return loss 
    
    def validation_step(self, batch, batch_nb):
        loss = torch.tensor(0.0, device="cuda:0") + torch.tensor(0.0, device="cuda:0")
        
        for i in range(self.num_tasks):
            x, y = batch["currency_" + str(i) + "_window"], batch["currency_" + str(i) + "_label"]

            output = self(x, i)
            #loss = F.nll_loss(output, y)
            loss += self.cross_entropy_loss[i](output, y)
 
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log("currency_" + str(i) +'_val_acc', acc, on_epoch=True, prog_bar=True, reduce_fx=torch.mean)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log("currency_" + str(i) +'_val_f1', f1, on_epoch=True, prog_bar=True, reduce_fx=torch.mean)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
    
    def test_step(self, batch, batch_nb):
        loss = torch.tensor(0.0, device="cuda:0") + torch.tensor(0.0, device="cuda:0")
        
        for i in range(self.num_tasks):
            x, y = batch["currency_" + str(i) + "_window"], batch["currency_" + str(i) + "_label"]

            output = self(x, i)
            print(y, torch.max(output, dim=1)[1])
            print(F.softmax(output)) # mantıken fark etmiyor
            #loss = F.nll_loss(output, y)
            loss += self.cross_entropy_loss[i](output, y)
            
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log("currency_" + str(i) +'_test_acc', acc, on_epoch=True, reduce_fx=torch.mean)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log("currency_" + str(i) +'_test_f1', f1, on_epoch=True, reduce_fx=torch.mean)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('test_loss', loss, on_epoch=True, reduce_fx=torch.mean)

        
    def configure_optimizers(self):
        
        optimizer = torch.optim.AdamW(model.parameters(), lr= 1e-3)#AdamW does weight decay
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

        return [optimizer], [{"scheduler": scheduler}]
    
    def train_dataloader(self):
        return self.train_dl

    def val_dataloader(self):
        return self.val_dl

    def test_dataloader(self):
        return self.test_dl

In [17]:
#logger = WandbLogger(name='lstm.v1',project='pytorchlightning')
logger = TensorBoardLogger("../output/models/lstm_model_logs", name="lstm_multi_task")

In [18]:
model = LSTM_based_classification_model()
trainer = pl.Trainer(gpus=-1, 
                     max_epochs= 150,
                     logger = logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [19]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type        | Params
----------------------------------------------------
0  | lstm_1             | LSTM        | 67.1 K
1  | batch_norm1        | BatchNorm2d | 256   
2  | lstm_2             | LSTM        | 132 K 
3  | batch_norm2        | BatchNorm2d | 256   
4  | lstm_3             | LSTM        | 132 K 
5  | batch_norm3        | BatchNorm2d | 256   
6  | dropout            | Dropout     | 0     
7  | linear1            | ModuleList  | 8.3 K 
8  | activation         | ReLU        | 0     
9  | output_layers      | ModuleList  | 195   
10 | cross_entropy_loss | ModuleList  | 0     
11 | f1_score           | F1          | 0     
12 | accuracy_score     | Accuracy    | 0     
----------------------------------------------------
340 K     Trainable params
0         Non-trainable params
340 K     Total params
1.362     Total estimated model params size (MB)
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_li

/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [20]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


tensor([0, 1, 2, 1, 2, 2, 1, 2, 2, 0, 0, 0, 2, 1, 0, 0], device='cuda:0') tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([[0.3377, 0.3581, 0.3041],
        [0.3367, 0.3560, 0.3074],
        [0.3373, 0.3572, 0.3055],
        [0.3379, 0.3571, 0.3050],
        [0.3373, 0.3560, 0.3067],
        [0.3386, 0.3572, 0.3042],
        [0.3363, 0.3550, 0.3087],
        [0.3369, 0.3569, 0.3061],
        [0.3386, 0.3575, 0.3039],
        [0.3361, 0.3562, 0.3077],
        [0.3371, 0.3577, 0.3053],
        [0.3377, 0.3582, 0.3041],
        [0.3374, 0.3593, 0.3033],
        [0.3361, 0.3555, 0.3084],
        [0.3372, 0.3560, 0.3068],
        [0.3369, 0.3565, 0.3066]], device='cuda:0')
tensor([0, 1, 2, 1, 2, 2, 1, 1, 1, 0, 0, 0, 0, 1, 2, 2], device='cuda:0') tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([[0.3377, 0.3590, 0.3033],
        [0.3366, 0.3567, 0.3067],
        [0.3371, 0.3581, 0.3048],
        [0.3379, 0.3579, 0.3043],
      

<ipython-input-16-56c20af0ae69>:156: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(output)) # mantıken fark etmiyor


[{'currency_0_test_acc': 0.25806450843811035,
  'currency_0_test_f1': 0.1367289274930954,
  'currency_1_test_acc': 0.4193548262119293,
  'currency_1_test_f1': 0.19648092985153198,
  'test_loss': 1.1045910120010376}]

In [23]:
for n in range(N_CURRENCIES):
    labels = [int(test_dataset[i]["currency_"+ str(n)+"_label"] )for i in range(test_dataset.__len__())]
    print(pd.DataFrame({"label": labels}).groupby("label").size())

label
0    12
1     8
2    11
dtype: int64
label
0     7
1    13
2    11
dtype: int64
